# Data Wrangling - Passing Yards

Couple of variable explanations

air_yards -- how many yards the ball is in the air before it reaches the target

EPA -- 'expected points added' This means, how does a given play impact the likelihood the offense will score. A 1st Down and 10 yards play at the 20 yard line with the offense completing a 20 yard pass has an EPA value of 1.36. The defensive EPA is just the inverse of this.

CPOE -- ''completion percentage over expected' This means, a calculation that considers air yards, field position, whether the quarterback was hit on the play, pass location, and many other variables. A high CPOE means that the quarterback is making difficult throws in the game. 

In [1]:
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

We wil import the nfl_data_py as our main database

In [3]:
import nfl_data_py as nfl

# Load NFL data for a specific range of years
data = nfl.import_pbp_data(range(2001, 2023))

# Print information about the DataFrame, including column names and data types
print(data.info())

2001 done.
2002 done.
2003 done.
2004 done.
2005 done.
2006 done.
2007 done.
2008 done.
2009 done.
2010 done.
2011 done.
2012 done.
2013 done.
2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
Downcasting floats.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057089 entries, 0 to 1057088
Columns: 390 entries, play_id to defense_coverage_type
dtypes: float32(206), int32(6), int64(1), object(177)
memory usage: 2.2+ GB
None


In [4]:
data.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,offense_players,defense_players,n_offense,n_defense,ngs_air_yards,time_to_throw,was_pressure,route,defense_man_zone_type,defense_coverage_type
0,1.0,2001_01_ATL_SF,2001090900,SF,ATL,REG,1,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,35.0,2001_01_ATL_SF,2001090900,SF,ATL,REG,1,SF,home,ATL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,57.0,2001_01_ATL_SF,2001090900,SF,ATL,REG,1,SF,home,ATL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,78.0,2001_01_ATL_SF,2001090900,SF,ATL,REG,1,SF,home,ATL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,99.0,2001_01_ATL_SF,2001090900,SF,ATL,REG,1,SF,home,ATL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We will see the column values in the database

In [5]:
data.columns.values

array(['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team',
       'season_type', 'week', 'posteam', 'posteam_type', 'defteam',
       'side_of_field', 'yardline_100', 'game_date',
       'quarter_seconds_remaining', 'half_seconds_remaining',
       'game_seconds_remaining', 'game_half', 'quarter_end', 'drive',
       'sp', 'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo',
       'ydsnet', 'desc', 'play_type', 'yards_gained', 'shotgun',
       'no_huddle', 'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble',
       'pass_length', 'pass_location', 'air_yards', 'yards_after_catch',
       'run_location', 'run_gap', 'field_goal_result', 'kick_distance',
       'extra_point_result', 'two_point_conv_result',
       'home_timeouts_remaining', 'away_timeouts_remaining', 'timeout',
       'timeout_team', 'td_team', 'td_player_name', 'td_player_id',
       'posteam_timeouts_remaining', 'defteam_timeouts_remaining',
       'total_home_score', 'total_away_score', 'posteam_score'

In [6]:
#This is a significantly large data set. We will find the number of columns we are dealing with:
num_columns = data.shape[1]
print(num_columns)

390


In [8]:
data.sample(10)

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,offense_players,defense_players,n_offense,n_defense,ngs_air_yards,time_to_throw,was_pressure,route,defense_man_zone_type,defense_coverage_type
350793,448.0,2008_08_WAS_DET,2008102603,DET,WAS,REG,8,DET,home,WAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1042394,2203.0,2022_14_KC_DEN,2022121110,DEN,KC,REG,14,DEN,home,KC,...,00-0036407;00-0036399;00-0029263;00-0037252;00...,00-0036374;00-0035625;00-0032210;00-0030511;00...,11.0,11.0,9.27,3.170,False,OUT,MAN_COVERAGE,COVER_0
128541,878.0,2003_14_CHI_GB,2003120700,GB,CHI,REG,14,GB,home,CHI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
830520,3036.0,2018_07_MIN_NYJ,2018102107,NYJ,MIN,REG,7,MIN,away,NYJ,...,00-0033960;00-0034270;00-0031588;00-0033909;00...,00-0031933;00-0026564;00-0029676;00-0031338;00...,11.0,11.0,-1.73,2.102,False,SCREEN,MAN_COVERAGE,COVER_1
123772,3433.0,2003_12_NO_PHI,2003112310,PHI,NO,REG,12,NO,away,PHI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
373876,229.0,2008_17_KC_CIN,2008122803,CIN,KC,REG,17,CIN,home,KC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
992199,1794.0,2021_14_LA_ARI,2021121300,ARI,LA,REG,14,None,None,None,...,,,0.0,0.0,NaN,NaN,None,None,None,None
103507,1765.0,2003_04_KC_BAL,2003092802,BAL,KC,REG,4,BAL,home,KC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121967,632.0,2003_11_STL_CHI,2003111611,CHI,LA,REG,11,LA,away,CHI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
976814,3562.0,2021_08_NE_LAC,2021103108,LAC,NE,REG,8,NE,away,LAC,...,00-0034960;00-0031852;00-0036972;00-0031580;00...,00-0035674;00-0035677;00-0034350;00-0034790;00...,11.0,11.0,NaN,NaN,None,None,None,None


In [7]:
data.tail(10)

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,offense_formation,offense_personnel,defenders_in_box,defense_personnel,number_of_pass_rushers,players_on_play,offense_players,defense_players,n_offense,n_defense
1057079,3947.0,2022_22_KC_PHI,2023021200,PHI,KC,POST,22,KC,away,PHI,...,SHOTGUN,"1 RB, 2 TE, 2 WR",6.0,"3 DL, 3 LB, 5 DB",NaN,46243;48516;47974;43367;46152;44874;40011;4618...,00-0034272;00-0032391;00-0034752;00-0033857;00...,00-0035386;00-0034993;00-0034375;00-0029653;00...,11.0,11.0
1057080,3979.0,2022_22_KC_PHI,2023021200,PHI,KC,POST,22,None,None,None,...,None,None,NaN,None,NaN,,,,0.0,0.0
1057081,3968.0,2022_22_KC_PHI,2023021200,PHI,KC,POST,22,KC,away,PHI,...,None,"1 RB, 3 TE, 1 WR",8.0,"3 DL, 3 LB, 5 DB",NaN,48516;46213;47974;43367;46152;40011;48683;4618...,00-0034386;00-0032391;00-0034752;00-0030506;00...,00-0035386;00-0034993;00-0034375;00-0029653;00...,11.0,11.0
1057082,4007.0,2022_22_KC_PHI,2023021200,PHI,KC,POST,22,None,None,None,...,None,None,NaN,None,NaN,,,,0.0,0.0
1057083,3996.0,2022_22_KC_PHI,2023021200,PHI,KC,POST,22,KC,away,PHI,...,None,"2 RB, 2 TE, 1 WR",6.0,"3 DL, 3 LB, 5 DB",NaN,48516;46213;47974;43367;46152;48683;46188;4132...,00-0034386;00-0032391;00-0034752;00-0035547;00...,00-0035386;00-0034993;00-0034375;00-0029653;00...,11.0,11.0
1057084,4035.0,2022_22_KC_PHI,2023021200,PHI,KC,POST,22,None,None,None,...,None,None,NaN,None,NaN,,,,0.0,0.0
1057085,4024.0,2022_22_KC_PHI,2023021200,PHI,KC,POST,22,KC,away,PHI,...,None,None,NaN,None,NaN,40994;46152;52618;54026;54478;53552;53492;5454...,00-0030615;00-0034752;00-0036230;00-0036623;00...,00-0036813;00-0037073;00-0036917;00-0037615;00...,11.0,11.0
1057086,4050.0,2022_22_KC_PHI,2023021200,PHI,KC,POST,22,PHI,home,KC,...,None,None,NaN,None,NaN,48259;54724;54568;54600;54026;53579;45102;4251...,00-0035157;00-0037198;00-0037819;00-0038043;00...,00-0036813;00-0036919;00-0033251;00-0036917;00...,11.0,11.0
1057087,4072.0,2022_22_KC_PHI,2023021200,PHI,KC,POST,22,PHI,home,KC,...,SHOTGUN,"1 RB, 1 TE, 3 WR",3.0,"0 DL, 4 LB, 7 DB",2.0,52608;52546;48259;46118;47846;43368;54568;5460...,00-0036271;00-0034351;00-0032954;00-0036919;00...,00-0036374;00-0035157;00-0035625;00-0037819;00...,11.0,11.0
1057088,4094.0,2022_22_KC_PHI,2023021200,PHI,KC,POST,22,None,None,None,...,None,None,NaN,None,NaN,,,,0.0,0.0


There seems to be a lot of missing data within this dataframe, we will need to see how much missing data we are dealing with. I will create a dataframe missing to see how many columns that are missing the majority of its data as we will not be using them later on.

In [9]:
#Finding missing values organized by the count and its percentage with filled in information
missing = pd.concat([data.isnull().sum(), 100 * data.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='%', ascending=False)

,count,%
drive_real_start_time,1057089,100.000000
lateral_sack_player_id,1057089,100.000000
lateral_sack_player_name,1057089,100.000000
tackle_with_assist_2_team,1057088,99.999905
tackle_with_assist_2_player_name,1057088,99.999905
...,...,...
quarter_end,0,0.000000
game_half,0,0.000000
surface,0,0.000000
div_game,0,0.000000


There are three columns with no data provided whatsoever, and there are multiple variables with a majority of NaN values.

In [10]:
#From the code above, there seems to be a lot of columns with a lot missing data. Lets see columns with more than 80% of its data missing:

missing_2 = pd.concat([data.isnull().sum(), 100 * data.isnull().mean()], axis=1)
missing_2.columns = ['count', '%']
missing_filtered = missing_2[missing_2['%'] >= 80.0]
missing_filtered.sort_values(by='%', ascending=False, inplace=True)
print(missing_filtered)

                                    count           %
lateral_sack_player_id            1057089  100.000000
drive_real_start_time             1057089  100.000000
lateral_sack_player_name          1057089  100.000000
tackle_with_assist_2_team         1057088   99.999905
tackle_with_assist_2_player_name  1057088   99.999905
...                                   ...         ...
assist_tackle_1_player_id          918402   86.880291
assist_tackle_1_team               918401   86.880196
assist_tackle_1_player_name        918401   86.880196
number_of_pass_rushers             911879   86.263219
yards_after_catch                  861775   81.523410

[114 rows x 2 columns]


/var/folders/2r/qvwb_49162s3q75v3pr4d1x00000gn/T/ipykernel_55443/407260728.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_filtered.sort_values(by='%', ascending=False, inplace=True)


In [11]:
missing_filtered.shape

(114, 2)

As shown above, there are 114 columns with over 80% of missing data, we have to decide how much to drop as they will not be used in the upcoming modeling process.

We will first drop the data that has over 90% of missing data.

There may be important variables in the missing_filtered dataframe, but I want to make sure there is no columns that may be relevant to the modeling process.

In [12]:
missing_3 = pd.concat([data.isnull().sum(), 100 * data.isnull().mean()], axis=1)
missing_3.columns = ['count', '%']
missing_filtered_2 = missing_3[missing_3['%'] >= 90.0]
missing_filtered_2.sort_values(by='%', ascending=False, inplace=True)
print(missing_filtered_2)

                                    count           %
drive_real_start_time             1057089  100.000000
lateral_sack_player_name          1057089  100.000000
lateral_sack_player_id            1057089  100.000000
tackle_with_assist_2_player_id    1057088   99.999905
tackle_with_assist_2_player_name  1057088   99.999905
...                                   ...         ...
tackle_with_assist_1_player_id     979597   92.669302
tackle_with_assist_1_team          979596   92.669208
tackle_with_assist_1_player_name   979596   92.669208
defense_man_zone_type              963254   91.123264
defense_coverage_type              963254   91.123264

[101 rows x 2 columns]


/var/folders/2r/qvwb_49162s3q75v3pr4d1x00000gn/T/ipykernel_55443/2260264742.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_filtered_2.sort_values(by='%', ascending=False, inplace=True)


Obviously, the columns where it has a 100% missing value percentage will be dropped. The columns that indicate over a value of 90% of missing data seem to be irrelevant to the upcoming modeling process.

In [13]:
# Displaying all variable names without truncation
pd.set_option('display.max_rows', None)
print(missing_filtered_2)

                                        count           %
drive_real_start_time                 1057089  100.000000
lateral_sack_player_name              1057089  100.000000
lateral_sack_player_id                1057089  100.000000
tackle_with_assist_2_player_id        1057088   99.999905
tackle_with_assist_2_player_name      1057088   99.999905
tackle_with_assist_2_team             1057088   99.999905
assist_tackle_3_team                  1057086   99.999716
assist_tackle_4_player_name           1057086   99.999716
assist_tackle_4_player_id             1057086   99.999716
assist_tackle_4_team                  1057086   99.999716
assist_tackle_3_player_name           1057086   99.999716
assist_tackle_3_player_id             1057086   99.999716
tackle_for_loss_2_player_name         1057078   99.998959
tackle_for_loss_2_player_id           1057078   99.998959
lateral_rusher_player_id              1057048   99.996121
lateral_rusher_player_name            1057048   99.996121
lateral_rushin

After determining which values that have over 90% of missing data, we will define a function that drops the columns with more than 90% of missing data, drop_columns_with_high_missing_data.
We will input the threshold as 90. 

In [14]:
#defining a function that will drop the columns with a certain percentage of missing data: in this case, 90% or more

def drop_columns_with_high_missing_data(data, threshold=90.0):
    """
    Drop columns from the DataFrame that have more than a specified percentage of missing data.

    Parameters:
    - df: DataFrame
    - threshold: float, optional (default=80.0)
        The threshold percentage for missing data. Columns with missing data exceeding this threshold will be dropped.

    Returns:
    - df_cleaned: DataFrame
        The DataFrame with columns having more than the specified percentage of missing data dropped.
    - dropped_columns: list
        List of column names that were dropped.
    """
    # Calculating the percentage of missing data for each column
    missing = pd.concat([data.isnull().sum(), 100 * data.isnull().mean()], axis=1)
    missing.columns = ['count', '%']

    # Filtering columns with missing data exceeding the threshold
    missing_filtered = missing[missing['%'] >= threshold]
    dropped_columns = missing_filtered.index.tolist()

    # Dropping columns from the DataFrame
    data_cleaned = data.drop(columns=dropped_columns)

    # Printing information about dropped columns to know which ones were dropped
    if dropped_columns:
        print(f"Dropped Columns ({len(dropped_columns)}): {dropped_columns}")
    else:
        print("No columns dropped.")

    return data_cleaned, dropped_columns


In [15]:
data_cleaned, dropped_columns = drop_columns_with_high_missing_data(data, threshold=90.0)
print(data_cleaned)   

Dropped Columns (101): ['field_goal_result', 'extra_point_result', 'two_point_conv_result', 'timeout_team', 'td_team', 'td_player_name', 'td_player_id', 'lateral_receiver_player_id', 'lateral_receiver_player_name', 'lateral_receiving_yards', 'lateral_rusher_player_id', 'lateral_rusher_player_name', 'lateral_rushing_yards', 'lateral_sack_player_id', 'lateral_sack_player_name', 'interception_player_id', 'interception_player_name', 'lateral_interception_player_id', 'lateral_interception_player_name', 'punt_returner_player_id', 'punt_returner_player_name', 'lateral_punt_returner_player_id', 'lateral_punt_returner_player_name', 'kickoff_returner_player_name', 'kickoff_returner_player_id', 'lateral_kickoff_returner_player_id', 'lateral_kickoff_returner_player_name', 'punter_player_id', 'punter_player_name', 'own_kickoff_recovery_player_id', 'own_kickoff_recovery_player_name', 'blocked_player_id', 'blocked_player_name', 'tackle_for_loss_1_player_id', 'tackle_for_loss_1_player_name', 'tackle_f

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
data_cleaned.columns.values

array(['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team',
       'season_type', 'week', 'posteam', 'posteam_type', 'defteam',
       'side_of_field', 'yardline_100', 'game_date',
       'quarter_seconds_remaining', 'half_seconds_remaining',
       'game_seconds_remaining', 'game_half', 'quarter_end', 'drive',
       'sp', 'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo',
       'ydsnet', 'desc', 'play_type', 'yards_gained', 'shotgun',
       'no_huddle', 'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble',
       'pass_length', 'pass_location', 'air_yards', 'yards_after_catch',
       'run_location', 'run_gap', 'kick_distance',
       'home_timeouts_remaining', 'away_timeouts_remaining', 'timeout',
       'posteam_timeouts_remaining', 'defteam_timeouts_remaining',
       'total_home_score', 'total_away_score', 'posteam_score',
       'defteam_score', 'score_differential', 'posteam_score_post',
       'defteam_score_post', 'score_differential_post', 'no_score_prob',


In [17]:
data_cleaned.shape

(1057089, 289)

Over a 100 columns have been dropped above.

In [30]:
data_cleaned.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,number_of_pass_rushers,players_on_play,offense_players,defense_players,n_offense,n_defense,ngs_air_yards,time_to_throw,was_pressure,route
0,1.0,2001_01_ATL_SF,2001090900,SF,ATL,REG,1,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,35.0,2001_01_ATL_SF,2001090900,SF,ATL,REG,1,SF,home,ATL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,57.0,2001_01_ATL_SF,2001090900,SF,ATL,REG,1,SF,home,ATL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,78.0,2001_01_ATL_SF,2001090900,SF,ATL,REG,1,SF,home,ATL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,99.0,2001_01_ATL_SF,2001090900,SF,ATL,REG,1,SF,home,ATL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


With the significantly large amounts of data with the multitude of variables, I will create a list of variables that will help me build my model that have possibly the most highest significance levels. 


In [18]:
columns = ['passer_player_name', 'posteam', 'defteam', 'season', 'week', 'home_team', 'away_team', 'play_type', 'air_yards', 
           'yards_after_catch', 'epa', 'complete_pass', 'incomplete_pass', 'interception', 'qb_hit', 'sack', 'pass_touchdown',
           'passing_yards', 'cpoe', 'roof', 'surface']

#None of these variables have been dropped
#We will create a new dataframe using only the columns 
qb_data = data[columns]

These will be the following features used for the upcoming regression model.

Passer Features:
QB passing completion
Pass Attempts
Air Yards
YAC
EPA
Interceptions
Sacks
QB Hits
Sacks
CPOE 
Pass Touchdowns


Defensive Features:
QB passing completion
Pass Attempts
Air Yards
YAC
EPA
Interceptions
Sacks
QB Hits
Sacks
CPOE 
Pass Touchdowns

Individual Features:
Field Surface
Roof

In [19]:
qb_data.head()

,passer_player_name,posteam,defteam,season,week,home_team,away_team,play_type,air_yards,yards_after_catch,...,complete_pass,incomplete_pass,interception,qb_hit,sack,pass_touchdown,passing_yards,cpoe,roof,surface
0,None,None,None,2001,1,SF,ATL,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,outdoors,grass
1,None,SF,ATL,2001,1,SF,ATL,kickoff,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,outdoors,grass
2,J.Garcia,SF,ATL,2001,1,SF,ATL,pass,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,6.0,NaN,outdoors,grass
3,None,SF,ATL,2001,1,SF,ATL,run,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,outdoors,grass
4,J.Garcia,SF,ATL,2001,1,SF,ATL,pass,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,26.0,NaN,outdoors,grass


Lets Check the datatypes for the dataframe qb_data to make sure we can go on for exploratory data analysis.

In [22]:
qb_data.dtypes

passer_player_name     object
posteam                object
defteam                object
season                  int64
week                    int32
home_team              object
away_team              object
play_type              object
air_yards             float32
yards_after_catch     float32
epa                   float32
complete_pass         float32
incomplete_pass       float32
interception          float32
qb_hit                float32
sack                  float32
pass_touchdown        float32
passing_yards         float32
cpoe                  float32
roof                   object
surface                object
dtype: object

It seems there are no problems and inconsistencies with the data types in the dataframe.

Lets see the types of play_types in this dataframe.

In [24]:
#Checking what sorts of play_types there are 
qb_data[['play_type']].sample(10)

,play_type
353389,pass
809470,pass
146014,pass
987052,run
314441,run
138202,pass
1043966,run
605257,field_goal
468566,pass
196987,extra_point


In [25]:
qb_data.shape

(1057089, 21)

There are over a million rows, but we only want to see the play_type as pass because we are predicting qb passing_yards.

We will filter the dataframe with the play_type as 'pass.'

In [34]:
# Dropping all rows that are not a pass because we are predicting qb_passing touchdowns
qb_data = qb_data[qb_data['play_type'] == 'pass']

In [35]:
qb_data[qb_data['play_type'] == 'pass'].sample(5)

,passer_player_name,posteam,defteam,season,week,home_team,away_team,play_type,air_yards,yards_after_catch,...,complete_pass,incomplete_pass,interception,qb_hit,sack,pass_touchdown,passing_yards,cpoe,roof,surface
310289,M.Schaub,HOU,NO,2007,11,HOU,NO,pass,4.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,5.0,23.475910,closed,grass
524796,M.Stafford,DET,SF,2012,2,SF,DET,pass,7.0,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,NaN,-54.946835,outdoors,grass
477931,E.Manning,NYG,LA,2011,2,NYG,LA,pass,7.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,7.0,36.172844,outdoors,fieldturf
579730,A.Smith,KC,NYG,2013,4,KC,NYG,pass,4.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,5.0,32.539551,outdoors,grass
310438,J.Kitna,DET,NYG,2007,11,DET,NYG,pass,0.0,6.0,...,1.0,0.0,0.0,0.0,0.0,0.0,6.0,38.476688,dome,fieldturf


We now have 21 columns with only the play_type as 'pass'

In [37]:
qb_data.shape

(426927, 21)

We now have 21 columns with 425,927 rows of data to go over

In [38]:
qb_data.head()

,passer_player_name,posteam,defteam,season,week,home_team,away_team,play_type,air_yards,yards_after_catch,...,complete_pass,incomplete_pass,interception,qb_hit,sack,pass_touchdown,passing_yards,cpoe,roof,surface
2,J.Garcia,SF,ATL,2001,1,SF,ATL,pass,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,6.0,NaN,outdoors,grass
4,J.Garcia,SF,ATL,2001,1,SF,ATL,pass,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,26.0,NaN,outdoors,grass
5,J.Garcia,SF,ATL,2001,1,SF,ATL,pass,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,outdoors,grass
7,J.Garcia,SF,ATL,2001,1,SF,ATL,pass,NaN,NaN,...,0.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,outdoors,grass
10,C.Chandler,ATL,SF,2001,1,SF,ATL,pass,NaN,NaN,...,0.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,outdoors,grass


In [40]:
# Grouping the data together by passer, week, season and aggregate
passing_df = qb_data.groupby(['passer_player_name', 'week', 'season'], as_index=False).agg(
    {'posteam' : 'first',
     'defteam' : 'first',
     'home_team' : 'first',
     'away_team' : 'first',
     'play_type' : 'first',
     'air_yards' : 'sum',
     'yards_after_catch' : 'sum',
     'epa' : 'sum',
     'complete_pass' : 'sum',
     'incomplete_pass' : 'sum',
     'interception' : 'sum',
     'qb_hit' : 'sum',
     'sack' : 'sum',
     'pass_touchdown' : 'sum',
     'passing_yards' : 'sum',
     'cpoe' : 'mean',
     'roof' : 'first',
     'surface' : 'first'
     }
)

In [41]:
#checking to see the format
qb_data.head()

,passer_player_name,posteam,defteam,season,week,home_team,away_team,play_type,air_yards,yards_after_catch,...,complete_pass,incomplete_pass,interception,qb_hit,sack,pass_touchdown,passing_yards,cpoe,roof,surface
2,J.Garcia,SF,ATL,2001,1,SF,ATL,pass,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,6.0,NaN,outdoors,grass
4,J.Garcia,SF,ATL,2001,1,SF,ATL,pass,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,26.0,NaN,outdoors,grass
5,J.Garcia,SF,ATL,2001,1,SF,ATL,pass,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,outdoors,grass
7,J.Garcia,SF,ATL,2001,1,SF,ATL,pass,NaN,NaN,...,0.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,outdoors,grass
10,C.Chandler,ATL,SF,2001,1,SF,ATL,pass,NaN,NaN,...,0.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,outdoors,grass


Saving the data into a csv file for the next EDA step.

In [42]:
#storing it so I can use it for the next process
qb_data.to_csv('passing_data_cleaned.csv', index=False)